# 2장 도커 명령어와 Dockerfile

- 구성
  - Docker에서 사용하는 용어
  - Docker 명령어
  - Dockerfile
  - Docker의 작업 흐름도
  - 자동화 빌드 구성

## 2.1 Docker에서 사용하는 용어

- Docker는 Git과 유사한 기능을 제공
  - commit: 인스턴스의 snapshopt 저장
  - 원격 저장소로 부터 이미지를 가져오고 저장하는 기능

### 2.1.1 Docker Container
 
- Docker Container
  - 호스트와 동일한 커널을 공유
  - 자원을 격리하는 환경 제공
  - 컨테이너 안에서 실행되는 애플리케이션은 언제나 동일한 환경 제공
- Docker Container의 파일 시스템
  - AUFS
      - 파일 시스템을 이루는 모든 레이어는 읽기 전용
      - 컨테이너 내부 프로세스는 모든 레이어가 함쳐진 파일 시스템을 볼 수 있음
      - 기본 레이어에 변경 발생 시 새로운 레이어 생성
      

### 2.1.2 Docker Daemon

- Docker Daemon
  - Docker Container를 관리하는 프로세스
- Docker Clint와 공통점과 차이점
  - 공통점: 동일한 바이너리 사용
  - 차이점: 실행시 Daemon은 Root권한 필요 

### 2.1.3 Docker Client

- Docker Contaienr를 관리하거나 실행하기 위해서 Docker Daemon과 상호작용하는 도구
- RESTFul API 사용

### 2.1.4 Dockerfile
- Docker Image 빌드 파일
- DSL로 작성됨
- GNU Makefile과 유사

### 2.1.5 Docker registry

- Docker Registory
  - Docker Image 저장소
- https://hub.docker.com/
  - Docker Inc

## 2.2 Docker Command

- Docker Command = Linux + git 스타일

### 2.2.1 Docker Daemon 실행

```bash
export DOCKER_HOST="tcp/0.0.0.0:2375"
docker -d -D -e lxc -s btrfs -dns 8.8.8.8 -dns-search example.com
```

- 서비스로 시작
```
sudo service docker start
```

- docker command의 flag
  - ```-d```
      - 데몬으로 실행
  - ```-D```
      - 디버깅 모드로 실행
  - ```-e [option]```
      - 더커 드라이버 지정
  - ```-s [option]```
      - 스토리지로 사용할 드라이버 지정
      - default: AUFS
  - ```--dns [options]```
      - Docker가 사용할 DNS 지정
  - ```--dns-search [options]```
      - DNS 탐색 서버 지정
  - ```-H [options]```
      - 데몬이 사용할 소켓 지정
      - 방식
        - ```tcp://host:port```
        - ```unix://path/to/socket```
        - ```fd://*```
        - ```fd://socketfd```

- Docker 명령 실행시 Docker Daemon 지정

```basj
docker -H tcp://0.0.0.0:2375 run -it ubuntu /bin/bash
```

- 위 명령은 다음과 동일한 명령
```bash
DOCKET_HOST="tcp://0.0.0.0:2375" docker run -it ubuntu /bin/bash
```

### 2.2.2 version

```
taewan@jupyter_docbase $docker -v
Docker version 1.12.1, build 6f9534c, experimental
taewan@jupyter_docbase $
```

### 2.2.3 info

```bash
taewan@jupyter_docbase $docker info
Containers: 2
 Running: 1
 Paused: 0
 Stopped: 1
Images: 7
Server Version: 1.12.1
Storage Driver: aufs
 Root Dir: /var/lib/docker/aufs
 Backing Filesystem: extfs
 Dirs: 119
 Dirperm1 Supported: true
Logging Driver: json-file
Cgroup Driver: cgroupfs
Plugins:
 Volume: local
 Network: host bridge null overlay
Swarm: inactive
Runtimes: runc
Default Runtime: runc
Security Options: seccomp
Kernel Version: 4.4.20-moby
Operating System: Alpine Linux v3.4
OSType: linux
Architecture: x86_64
CPUs: 4
Total Memory: 1.952 GiB
Name: moby
ID: NQFI:QJRH:VBCG:JGJM:LZ2W:DRJM:74FF:RNCA:DRKL:2UIX:X2VB:5UWZ
Docker Root Dir: /var/lib/docker
Debug Mode (client): false
Debug Mode (server): true
 File Descriptors: 21
 Goroutines: 34
 System Time: 2016-09-26T06:52:46.074203923Z
 EventsListeners: 1
No Proxy: *.local, 169.254/16
Username: taewanme
Registry: https://index.docker.io/v1/
Experimental: true
Insecure Registries:
 127.0.0.0/8
taewan@jupyter_docbase $ 
```


### 2.2.4 run

- 형식: docker run [option] [docker_inage] [명령어] [인자]

#### option
- ```-a, --attach=[]```: stdin, stdout, stderr 파일을 연결
- ```-d, --detach ```: 백그라운드 실행
- ```-i, --interactive ```: 컨테이너에 연결된 표준 출력을 유지
- ```-t, --tty ```
  - 가상 터미널 활성화 
  - 터미널 직접 연결시 사용
- ```-p, --publish=[] ```
  - port 연결
  - ip:hostport:contaienrport
- ```--rm ```
  - 실행 종료시 자동 컨테이너 삭제
  - ```-d```와 함께 사용 불가
- ```--privileged ```
  - 추가 권한 부여
  - 커널 작업시 사용
- ```-v, --volume ```: 볼륨할당
  - :ro
    - 읽기전용
  - :rw
    - 읽기 쓰기
- ```-w, --workdir="" ```: 실행 디렉터리 지정
- ```--name="" ```: 컨테이너 명 지정
- ```-h, --hostname="" ```
- ```-u, --user="" ```: user 명 및 uid 지정
- ```-e, --env="" ```: 컨테이너 안에서 사용할 환경 변수 지정
- ```--env-file=[] ```
- ```--dns=[] ```
- ```--dns-search=[] ```
- ```--link=[] ```: 다른 컨테이너와 네트워크와 환경 변수 공유시 사용
- ```-c, --cpu-shares=0 ```:  2 ~ 1024,  cpu 점유 우선순위
- ```--cpuset="" ```
  - 사용할 cpu 지정
  - 지정된 cpu만 사용
  - 0부터 시작
- ```-m, --memory="" ```
  - 메모리 사영량 제한
  - 단위
    - b, m,g
- ```--restart="" ```
  - 알수 없는 종료시 재시작 옵션
  - 컨테이너 재시작 정책 설정
    - no: 기본값, 재시작 없음
    - on-failure: 종료코드가 0이 아닌 경우 재시작
      - 재시작 횟수 지정
        - on-failure:5
    - always: 항상 재시작
- ```--cap-add="" ```
  - 특정 권한 부여
- ```--cap-drop=""```
  - 특정 권한 삭제
- ```--device=""```: 특정 장치 마운트
  - ```--privileged```함께 사용 가능
  - ```--device=/dev/video0:/dev/video0
  



```
> docker run -v ${pwd}:${pwd} -w ${pwd} -p 0.0.0.0:8000:8000 \ 
-it --rm --name server \ 
python:2.7 python -m SimpleHTTPServer 8000;

```

### 2.2.5 search

```
> docker search python | less
```



### 2.2.6 pull

- 형식: docker pull <registry_경로><이미지>


### 2.2.7 start

- 종료한 상태의 컨테이너 시작하기
- run과 동일한 플래그 사용 가능

> docker start -i -a [컨테이너1, 컨테이너2..]

### 2.2.8 stop

- SIGTERM singnal과 SIGKILL 신호를 현재 실행하는 컨테이너에 보냄
  - SIGTERM: 프로세스들 동작 중지
  - SIGKILL: 강제로 프로세스 종료

### 2.2.9 restart

- 현재 실행 중인 컨테이너 재시작

### 2.2.10 rm

- Docker 컨테이너 삭제

- tip
  - 모든 컨테이너 이미지 삭제
  - ```docker rm $(docker ps -a -q)```

## 2.2.11 ps

- ```-a, --a```
  - 멈춰있는 컨테이너 포함
- ```-q, --quiet```
  - 컨테이너 ID
- ```-s, --size```
  - 컨테이너 크기
- ```-l, --last```
  - 가장 최근 컨테이너만 표시
- ```-n=""```
  - 최근 몇개만 표시
- ```--before=""```
  = 특정 ID 이전 컨테이너만 실행
- ```--after=""```
  - 특정 ID 기준으로 이후에 생성된 

### 2.2.13 inspect

- 내부 정보 공개


```
taewan@jupyter_docbase $docker inspect jupyter
[
    {
        "Id": "cdf357309c8a081b373c2d9826ab404eefbc515f314acb1a4da28488e8d1977e",
        "Created": "2016-09-25T05:19:58.12621668Z",
        "Path": "tini",
        "Args": [
            "--",
            "startJupyter.sh"
        ],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 2408,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2016-09-25T05:19:58.961200351Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:8b7cd81ccf75da908087aa1782e9ec0ab3e79fd4ff6ea67e268864ee4cf5e2a7",
        "ResolvConfPath": "/var/lib/docker/containers/cdf357309c8a081b373c2d9826ab404eefbc515f314acb1a4da28488e8d1977e/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/cdf357309c8a081b373c2d9826ab404eefbc515f314acb1a4da28488e8d1977e/hostname",
        "HostsPath": "/var/lib/docker/containers/cdf357309c8a081b373c2d9826ab404eefbc515f314acb1a4da28488e8d1977e/hosts",
        "LogPath": "/var/lib/docker/containers/cdf357309c8a081b373c2d9826ab404eefbc515f314acb1a4da28488e8d1977e/cdf357309c8a081b373c2d9826ab404eefbc515f314acb1a4da28488e8d1977e-json.log",
        "Name": "/jupyter",
        "RestartCount": 0,
        "Driver": "aufs",
        "MountLabel": "",
        "ProcessLabel": "",
        "AppArmorProfile": "",
        "ExecIDs": null,
        "HostConfig": {
            "Binds": [
                "/Users/taewan/docu_root/jupyter_docbase:/home/jovyan/work/data"
            ],
            "ContainerIDFile": "",
            "LogConfig": {
                "Type": "json-file",
                "Config": {}
            },
            "NetworkMode": "default",
            "PortBindings": {
                "8888/tcp": [
                    {
                        "HostIp": "",
                        "HostPort": "8888"
                    }
                ]
            },
            "RestartPolicy": {
                "Name": "no",
                "MaximumRetryCount": 0
            },
            "AutoRemove": false,
            "VolumeDriver": "",
            "VolumesFrom": null,
            "CapAdd": null,
            "CapDrop": null,
            "Dns": [],
            "DnsOptions": [],
            "DnsSearch": [],
            "ExtraHosts": null,
            "GroupAdd": null,
            "IpcMode": "",
            "Cgroup": "",
            "Links": null,
            "OomScoreAdj": 0,
            "PidMode": "",
            "Privileged": false,
            "PublishAllPorts": false,
            "ReadonlyRootfs": false,
            "SecurityOpt": null,
            "UTSMode": "",
            "UsernsMode": "",
            "ShmSize": 67108864,
            "Runtime": "runc",
            "ConsoleSize": [
                0,
                0
            ],
            "Isolation": "",
            "CpuShares": 0,
            "Memory": 0,
            "CgroupParent": "",
            "BlkioWeight": 0,
            "BlkioWeightDevice": null,
            "BlkioDeviceReadBps": null,
            "BlkioDeviceWriteBps": null,
            "BlkioDeviceReadIOps": null,
            "BlkioDeviceWriteIOps": null,
            "CpuPeriod": 0,
            "CpuQuota": 0,
            "CpusetCpus": "",
            "CpusetMems": "",
            "Devices": [],
            "DiskQuota": 0,
            "KernelMemory": 0,
            "MemoryReservation": 0,
            "MemorySwap": 0,
            "MemorySwappiness": -1,
            "OomKillDisable": false,
            "PidsLimit": 0,
            "Ulimits": null,
            "CpuCount": 0,
            "CpuPercent": 0,
            "IOMaximumIOps": 0,
            "IOMaximumBandwidth": 0
        },
        "GraphDriver": {
            "Name": "aufs",
            "Data": null
        },
        "Mounts": [
            {
                "Source": "/Users/taewan/docu_root/jupyter_docbase",
                "Destination": "/home/jovyan/work/data",
                "Mode": "",
                "RW": true,
                "Propagation": "rprivate"
            }
        ],
        "Config": {
            "Hostname": "cdf357309c8a",
            "Domainname": "",
            "User": "jovyan",
            "AttachStdin": false,
            "AttachStdout": false,
            "AttachStderr": false,
            "ExposedPorts": {
                "8888/tcp": {}
            },
            "Tty": true,
            "OpenStdin": true,
            "StdinOnce": false,
            "Env": [
                "no_proxy=*.local, 169.254/16",
                "PATH=/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin",
                "DEBIAN_FRONTEND=noninteractive",
                "CONDA_DIR=/opt/conda",
                "SHELL=/bin/bash",
                "NB_USER=jovyan",
                "NB_UID=1000",
                "HOME=/home/jovyan",
                "LC_ALL=en_US.UTF-8",
                "LANG=en_US.UTF-8",
                "LANGUAGE=en_US.UTF-8"
            ],
            "Cmd": [
                "startJupyter.sh"
            ],
            "Image": "taewanme/jupyter:1.0",
            "Volumes": {
                "/home/jovyan/work/data": {}
            },
            "WorkingDir": "/home/jovyan/work",
            "Entrypoint": [
                "tini",
                "--"
            ],
            "OnBuild": null,
            "Labels": {}
        },
        "NetworkSettings": {
            "Bridge": "",
            "SandboxID": "d5bd81742f98f56118c16a30ac9acbdf2747f31efdd1f5ff2e10175aa09466db",
            "HairpinMode": false,
            "LinkLocalIPv6Address": "",
            "LinkLocalIPv6PrefixLen": 0,
            "Ports": {
                "8888/tcp": [
                    {
                        "HostIp": "0.0.0.0",
                        "HostPort": "8888"
                    }
                ]
            },
            "SandboxKey": "/var/run/docker/netns/d5bd81742f98",
            "SecondaryIPAddresses": null,
            "SecondaryIPv6Addresses": null,
            "EndpointID": "3e845c7f08466562da10e3a705455827075f525a8014df0500a3f44be660de18",
            "Gateway": "172.17.0.1",
            "GlobalIPv6Address": "",
            "GlobalIPv6PrefixLen": 0,
            "IPAddress": "172.17.0.2",
            "IPPrefixLen": 16,
            "IPv6Gateway": "",
            "MacAddress": "02:42:ac:11:00:02",
            "Networks": {
                "bridge": {
                    "IPAMConfig": null,
                    "Links": null,
                    "Aliases": null,
                    "NetworkID": "9ac4460c78e8dfefb005f5debfdfa971c069ac151e18d984f4d3272fca9c87f7",
                    "EndpointID": "3e845c7f08466562da10e3a705455827075f525a8014df0500a3f44be660de18",
                    "Gateway": "172.17.0.1",
                    "IPAddress": "172.17.0.2",
                    "IPPrefixLen": 16,
                    "IPv6Gateway": "",
                    "GlobalIPv6Address": "",
                    "GlobalIPv6PrefixLen": 0,
                    "MacAddress": "02:42:ac:11:00:02"
                }
            }
        }
    }
]

```

- 데이터 검색

```
taewan@jupyter_docbase $docker inspect  -f '{{.NetworkSettings.IPAddress}}' jupyter;
172.17.0.2
```

- go 템플릿을 이용한 검색
```
taewan@jupyter_docbase $docker inspect  -f '{{.NetworkSettings}}' jupyter;
{{ d5bd81742f98f56118c16a30ac9acbdf2747f31efdd1f5ff2e10175aa09466db false  0 map[8888/tcp:[{0.0.0.0 8888}]] /var/run/docker/netns/d5bd81742f98 [] []} {3e845c7f08466562da10e3a705455827075f525a8014df0500a3f44be660de18 172.17.0.1  0 172.17.0.2 16  02:42:ac:11:00:02} map[bridge:0xc4203de000]}
```


### 2.2.14 top

```bash
taewan@jupyter_docbase $docker top jupyter
PID                 USER                TIME                COMMAND
2408                dockrema            0:19                tini -- startJupyter.sh
2433                dockrema            0:00                {startJupyter.sh} /bin/bash /usr/local/bin/startJupyter.sh
2434                dockrema            3:43                {jupyter-noteboo} /opt/conda/bin/python /opt/conda/bin/jupyter-notebook --notebook-dir=./data
2452                dockrema            1:24                /opt/conda/bin/python -m ipykernel -f /home/jovyan/.local/share/jupyter/runtime/kernel-81610122-d867-4995-b190-02b31825be63.json
2479                dockrema            0:26                /opt/conda/bin/python -m ipykernel -f /home/jovyan/.local/share/jupyter/runtime/kernel-db825b3b-61e5-4c9e-89ee-3bb4dc90e937.json
2995                dockrema            1:05                /opt/conda/bin/python -m ipykernel -f /home/jovyan/.local/share/jupyter/runtime/kernel-73c4d97b-828a-4aa5-ad80-f6c7820777b4.json
taewan@jupyter_docbase $
```

### 2.2.15 attach
### 2.2.16 kill
 
- 컨테이너에 SIGTERM을 날림

### 2.2.17 cp
- docker cp jupyter:/post/file_name:/contaier/path
### 2.2.18 port
```
taewan@jupyter_docbase $docker port jupyter
8888/tcp -> 0.0.0.0:8888
You have new mail in /var/mail/taewan
taewan@jupyter_docbase $
```

### 2.2.19 diff
- base image와의 차이점 기술

### 2.3 image 관련 명령

- images
- rmi
- save
```
docker save <tar file name> <container_name>
```
- load
  - tarball로 부터 로딩
  ```
    docker load -i ./container.tar
  ```

- export
  - 컨테이너의 파일 시스템을 tarball 평태로 저장
  ```
    docker export <container_name> >> tarball.name
  ```

- import
  - 비어있는 파일시스템을 만들어 tarball로 부터 파일시스템을 가져옴
- tag
  - 버전 추가
- login
- history
- push
- build
  - docker build options <경로>
  - options
    - --no-cache
    - --force-rm
    - --rm=true
    - -t, --tag-""
```
docker build -t taewan/jupyter:2.0 .
```

## dockerfile

### comment
- ```#```

### FROM
- base image 지정
- ```FROM <image>:<tag>
- tag 미지정시:latest

### MAINTAINER
- ```MAINTAINER <name>```

### RUN
- 이미지를 빌드하며 실행할 명령
- ```RUN <실행명령>```
- ```RUN ["실행명령","인자1","인자2"]```
- ```/bin/bash -c <실행명령>``` 형태로 수행
  - -c옵션은 string으로 부터 수행하는 옵션
  
### CMD
- ```CMD ["실행명령","인자1","인자2"]```
- ```CMD <실행 명령>```

### ENTRYPOINT
- CMD와 함께 사용할 수 있음

### WORKDIR

### EXPOSE

### ENV
- ```ENV <키>=<값>```
- 환경 변수 등록

### USER
- ```USER <user>

### VOLUME
``` VOLUME ['/data','/dataw']```

### ADD
- ```ADD <origin_file> <targret_dir>```
- origin_file은 빌드 컨텍스트 안에 있어야 함
  - ../file_name 처럼 컨텍스트를 벗어난 지정 불가
  
### onbuild
- 기본이미지로 사용될 때 실행할 크리거
- FROM, MAINTAINER, ONBUILD 제외한 모든 항목에 사용 가능



## 2.6 빌드 자동화 구성
- Automated Build 등록
![](./images/step01.jpeg)
----
- github 혹은 Bitbucket 등록
![](./images/step02.jpeg)
----
- github repository 선택
![](./images/step03.jpeg)
----
- docker registry의 repository 기본 정보 등록
![](./images/step04.jpeg)
----
- github의 대상 dockerfile 위치 지정 및 tag 정보 등록
![](./images/step05.jpeg)
----
- 최종 repository 생성
![](./images/step06.jpeg)
